# Welcome to the SWI-Prolog kernel
Using this kernel you can run SWI Prolog code in a jupyter notebook. Each cell can be placed into a consultation file by writing `%file: <filename>.pl` at the top of the cell. These consultation files are neccessary as otherwise the kernel defaults to using `cell.pl` as the consultation file - meaning cells may overwrite one another!

In [1]:
%file: foo.pl

man(socrates).
mortal(X) :- man(X).

In [2]:
%file: bar.pl

cat(crookshanks).
animal(X) :- cat(X).

Even though the above Prolog is in different cells, because they reference different consultation files both are added to the factbase!

We can see this by querying the facts asserted in the cells. Queries are executed by prefixing with `?-`

In [3]:
?- mortal(socrates).
?- mortal(X).
?- mortal(bob).

?- cat(crookshanks).
?- cat(X).
?- animal(X).

true.
X = socrates .
false.
true.
X = crookshanks .
X = crookshanks .

---

This kernel also integrates the Hyperledger Iroha project. Specifically, this integration allows for consultation files to be hashed and stored on the blockchain, meaning we can track when a certain consultation file is updated and verify that a consultation file is the most recent version by comparing hashes. More information on this project can be found in the GitHub repository: https://github.com/Gamma749/SWIPL-Notebook-Blockchain-Filehashing

To better support this integration, several variables can affect the behavior of the kernel and the blockchain interactions

- `BLOCKCHAIN` determines if the kernel should log file hashes onto the blockchain at all. 
- `TIMSTAMPING` determines if a consultation file should have a timestamp added to it, meaning an unchanged file can be hashed multiple times onto the ledger
- `LOGGING_LEVEL` does not specifically relate to the blockchain, instead this determines the level of logging that is output to the terminal. Useful for debugging and development!

We can change these variables within the notebook using the magic `%ENV` tag. Let's investigate blockchain integration by changing this variable to `1`.


In [4]:
%ENV
BLOCKCHAIN=1
TIMESTAMPING=1
LOGGING_LEVEL=20

SET ENV BLOCKCHAIN=1
SET ENV TIMESTAMPING=1
SET ENV LOGGING_LEVEL=20

Now we can run Prolog cells again and see new output:

In [9]:
%file: foo.pl

man(socrates).
mortal(X) :- man(X).

File: foo.pl
Timestamp: 1639100583364621163
Hash: 1148e073e5c464a0dc55c2c8a9e5b55e
Domain: swipluser-foo-hash
Iroha Response: COMMITTED

The output here is some information on what has been sent to the blockchain.
- File: The name of the file we have hashed
- Timestamp: The timestamp of when the file was run
- Hash: The MD5 hash of the file
- Domain: The Iroha domain the file is stored on (for ease of tracking specific files)
    - Note that the swipluser part of the domain derives from the Iroha username generated when this container started!
- Iroha Response: The response of the blockchain, if the file was commited or not

Try running the cell again, and see the timestamp and hash change!

---
With further hacks, this kernel also supports python scripting using `%python` magic!

In [6]:
%python
print("Hello, World!")

Hello, World!

Note that this is very delicate and not intended to be a production python environment. If needed, the Python3 kernel is also available on this container. However, this allows us to perform basic pyhton scripting within a prolog notebook.

For example, we can load up our Iroha utilities and investigate our file hashes from above!

In [11]:
%python
from IrohaUtils import *
import json
custodian = IrohaHashCustodian.Custodian()

hashes = custodian.get_domain_hashes("swipluser-foo-hash")
for hash in hashes: print(json.dumps(hash, indent=2))

{
  "hash": "0eca545f014bf62ef7f12077d5ada0c4",
  "height": 6,
  "domain": "swipluser-foo-hash",
  "creator_id": "swipluser@hashing",
  "time": 1639100561041
}
{
  "hash": "25ac788b5f01c6a17369af7184d879ed",
  "height": 9,
  "domain": "swipluser-foo-hash",
  "creator_id": "swipluser@hashing",
  "time": 1639100579180
}
{
  "hash": "1148e073e5c464a0dc55c2c8a9e5b55e",
  "height": 11,
  "domain": "swipluser-foo-hash",
  "creator_id": "swipluser@hashing",
  "time": 1639100585281
}

And now we can see the hashes of files generated above!

![Image](images/prolog-image.png)